In [ ]:
!RD /S /Q data\all_data
!python main.py -num-tasks 1000 -num-classes 5 -num-dim 60 -seed 1 --prob-clusters 0.2 0.3 0.3 0.2

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import json

with open("data/all_data/data.json") as f:
    data = json.load(f)
    f.close()

In [ ]:
def fit_score(df, cluster=None):
    X, y = df.iloc[:,:-3], df.iloc[:,-3]
    logreg = LogisticRegression(
        max_iter = 2000,
        multi_class = 'multinomial',
    )
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    logreg.fit(X,y)
    if cluster is not None:
        print("Accuracy for cluster", str(cluster) + ":", np.round(logreg.score(X,y),3))
    else:
        print("Accuracy for all samples,")
        print("  not seperated by tasks:", np.round(logreg.score(X,y),3))

In [ ]:
users = data["users"]

## make a pandas df from the json with X,y,user_id,cluster_id
try:
    del df
except NameError:
    pass
for user in users:
    cluster = data["cluster_ids"][int(user)]
    X = data["user_data"][user]["x"]
    y = data["user_data"][user]["y"]
    df_temp = pd.DataFrame(list(map(lambda a: tuple(a[0] + [a[1]]), zip(X,y))))
    df_temp["user_id"] = int(user)
    df_temp["cluster_id"] = int(cluster)
    new_columns = df_temp.columns[:-3]
    new_columns = list(map(lambda x: "x_" + str(x), new_columns)) + ["y", "user_id", "cluster_id"]
    df_temp.columns = new_columns
    try:
        df = df.append(df_temp, ignore_index=True)
    except NameError:
        df = df_temp

In [ ]:
unique_classes = list(df.y.unique())
unique_cluster = list(df.cluster_id.unique())
input_dimensions = df.shape[1] - 3
samples = df.shape[0]

print("Number of Task Cluster:", str(len(unique_cluster)) + ", Number of Classes:", str(len(unique_classes)))
print("Input Dimensions:", str(input_dimensions) + ", Samples in Total:", str(samples))
print()
print("Samples per Cluster:")
cluster_distribution = zip(list(df.cluster_id.value_counts().index),
                            list(df.cluster_id.value_counts(normalize=False)),
                            list(df.cluster_id.value_counts(normalize=True)))
print("id, total, relative")
for cd in sorted(cluster_distribution):
    print(cd[0], cd[1], np.round(cd[2],3))

print()
print("Samples per Class:")
class_distribution = zip(list(df.y.value_counts().index),
                        list(df.y.value_counts(normalize=False)),
                        list(df.y.value_counts(normalize=True)))
print("id, total, relative")
for cd in sorted(class_distribution):
    print(cd[0], cd[1], np.round(cd[2],3))
print()

### TRAINING IS HAPPENING HERE ###

fit_score(df)
for cl in sorted(unique_cluster):
    df_cl = df.loc[df.cluster_id == cl]
    fit_score(df_cl, cl)